In [281]:
import pandas as pd
import numpy as np



In [282]:
train = pd.read_csv('./Data/train_cleaned.csv')
test = pd.read_csv('./Data/test_cleaned.csv')
target = pd.read_csv('./Data/training_set_labels.csv')

#total = pd.concat([train,test])
total = train.append(test, ignore_index = True)

In [283]:
target.head(1)

,id,status_group
0,69572,functional


In [284]:
train = train.drop(['id'], axis = 1) #Dropping the id from the train dataset as it will not help in any way in predicting the target
test_ids = test['id'] #Keeping the test ids in a separate file for final prediction
test = test.drop(['id'], axis = 1) #Again dropping the ids from the test dataset as they are not going to help in final prediction
target = target.drop(['id'], axis = 1) #Similarly dropping the ids from the train labels dataset

In [285]:
target['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [286]:
proportion = (target['status_group'].value_counts()/target['status_group'].value_counts().sum())
proportion

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [287]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 29 columns):
Unnamed: 0           74250 non-null int64
id                   74250 non-null int64
amount_tsh           74250 non-null float64
funder               74250 non-null object
gps_height           74250 non-null float64
installer            74250 non-null object
longitude            74250 non-null float64
latitude             74250 non-null float64
basin                74250 non-null object
subvillage           73780 non-null object
region               74250 non-null object
region_code          74250 non-null int64
district_code        74250 non-null int64
lga                  74250 non-null object
ward                 74250 non-null object
population           74250 non-null float64
public_meeting       70095 non-null object
scheme_management    69404 non-null object
scheme_name          74250 non-null object
permit               70457 non-null object
construction_year    74250 non

In [288]:
total.columns

Index(['Unnamed: 0', 'id', 'amount_tsh', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'management', 'payment_type', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'operation_years'],
      dtype='object')

In [289]:
total = total.drop(['funder','scheme_name','subvillage','region','region_code','district_code','lga','ward',
              'public_meeting','water_quality'], axis = 1)

In [290]:
total['population'] = total['population'].fillna(total['population'].median())
if total['construction_year'].value_counts().index[0]==0:
    total['construction_year'] = total['construction_year'].fillna(total['construction_year'].value_counts().index[1])
else:
    total['construction_year'] = total['construction_year'].fillna(total['construction_year'].value_counts().index[0])

In [291]:
cols = ['installer','scheme_management','permit']
for col in cols:
    if total[col].value_counts().index[0]==0:
        total[col] = total[col].fillna(total[col].value_counts().index[1])
    else:
        total[col] = total[col].fillna(total[col].value_counts().index[0])

In [292]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 19 columns):
Unnamed: 0           74250 non-null int64
id                   74250 non-null int64
amount_tsh           74250 non-null float64
gps_height           74250 non-null float64
installer            74250 non-null object
longitude            74250 non-null float64
latitude             74250 non-null float64
basin                74250 non-null object
population           74250 non-null float64
scheme_management    74250 non-null object
permit               74250 non-null bool
construction_year    74250 non-null float64
extraction_type      74250 non-null object
management           74250 non-null object
payment_type         74250 non-null object
quantity             74250 non-null object
source               74250 non-null object
waterpoint_type      74250 non-null object
operation_years      74250 non-null int64
dtypes: bool(1), float64(6), int64(3), object(9)
memory usage: 10.3+ MB


In [293]:
#divide data again into train and test

train = total.iloc[:59400,:]
test = total.iloc[59400:,:]

In [294]:
#installer dataframe & dummified target
train_installer_target = pd.concat([train['installer'],target], axis=1)
target_dummies = pd.get_dummies(train_installer_target['status_group'])
train_installer_target = train_installer_target.drop(['status_group'], axis = 1)
train_installer_target = pd.concat([train_installer_target,target_dummies], axis=1)
train_installer_target.head(2)

,installer,functional,functional needs repair,non functional
0,DWE,1,0,0
1,DWE,1,0,0


In [295]:
train_installer_target_prop = pd.concat([train_installer_target.groupby('installer').mean(),train_installer_target.groupby('installer').count()], axis = 1).iloc[:,:-2]
train_installer_target_prop.columns = ['functional','functional needs repair','non functional','count']
train_installer_target_prop.sort_values(by='count',ascending=False).head()

,functional,functional needs repair,non functional,count
installer,,,,
Others,0.539376,0.073783,0.386841,33802
DWE,0.545225,0.072750,0.382025,17402
Government,0.557808,0.065205,0.376986,1825
RWE,0.537313,0.072968,0.389718,1206
Commu,0.547170,0.065094,0.387736,1060


In [296]:
train_installer_target_prop['category'] = np.zeros((train_installer_target_prop.shape[0],1))
train_installer_target_prop['category'].loc[train_installer_target_prop['functional'] > 0.54] = 'functional'
train_installer_target_prop['category'].loc[train_installer_target_prop['non functional'] > 0.38] = 'non functional'
train_installer_target_prop['category'].loc[train_installer_target_prop['functional needs repair'] > 0.05] = 'functional needs repair'

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [297]:
train['installer_category'] = train.installer.map(train_installer_target_prop.category)
test['installer_category'] = test.installer.map(train_installer_target_prop.category)
test['installer_category'] = test['installer_category'].fillna(test['installer_category'].value_counts().index[0])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [298]:
total = train.append(test, ignore_index = True)

In [299]:
total = total.drop(['installer'], axis=1)

In [300]:
total.describe(include = ['O'])

,basin,scheme_management,extraction_type,management,payment_type,quantity,source,waterpoint_type,installer_category
count,74250,74250,74250,74250,74250,74250,74250,74250,74250
unique,9,12,18,12,7,5,10,7,1
top,Lake Victoria,VWC,gravity,vwc,never pay,enough,spring,communal standpipe,functional needs repair
freq,12871,50763,33263,50624,31712,41522,21216,35628,74250


In [301]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 19 columns):
Unnamed: 0            74250 non-null int64
id                    74250 non-null int64
amount_tsh            74250 non-null float64
gps_height            74250 non-null float64
longitude             74250 non-null float64
latitude              74250 non-null float64
basin                 74250 non-null object
population            74250 non-null float64
scheme_management     74250 non-null object
permit                74250 non-null bool
construction_year     74250 non-null float64
extraction_type       74250 non-null object
management            74250 non-null object
payment_type          74250 non-null object
quantity              74250 non-null object
source                74250 non-null object
waterpoint_type       74250 non-null object
operation_years       74250 non-null int64
installer_category    74250 non-null object
dtypes: bool(1), float64(6), int64(3), object(9)
memor

In [302]:
total_dummies = pd.get_dummies(total)

In [303]:
total_dummies.head()

,Unnamed: 0,id,amount_tsh,gps_height,longitude,latitude,population,permit,construction_year,operation_years,...,source_spring,source_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,installer_category_functional needs repair
0,325,21669,777.777778,1715.0,31.366025,-7.972010,1.0,True,1974.0,37,...,1,0,0,1,0,0,0,0,0,1
1,11364,60108,1352.941176,1906.0,31.431333,-7.850336,1.0,True,1976.0,35,...,0,0,0,1,0,0,0,0,0,1
2,14462,44820,777.777778,1741.0,31.368317,-7.971081,1.0,True,1974.0,37,...,1,0,0,1,0,0,0,0,0,1
3,18159,53307,777.777778,1730.0,31.368730,-7.972788,1.0,True,1974.0,37,...,1,0,0,1,0,0,0,0,0,1
4,18990,5701,777.777778,1731.0,31.365461,-7.970985,1.0,True,1974.0,37,...,1,0,0,1,0,0,0,0,0,1


In [305]:
total_dummies = total_dummies.drop(['construction_year'], axis=1)
total_dummies.shape

(74250, 90)

In [306]:
train = total_dummies.iloc[:59400,:]
test = total_dummies.iloc[59400:,:]

In [307]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [308]:
classifier = RandomForestClassifier(n_estimators=500, random_state = 8)


In [309]:
score = cross_val_score(classifier, train, np.ravel(target), scoring = 'accuracy',cv=5).mean()
print("Score: {}".format(score))

Score: 0.512205534712671


In [311]:
model = RandomForestClassifier(n_estimators=500, random_state=8)
model.fit(train,np.ravel(target))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=8, verbose=0, warm_start=False)

In [312]:
test_predictions = model.predict(test)

In [313]:
submission = pd.DataFrame(test_predictions, columns=['status_group'])
submission.insert(0, 'id', test_ids)
submission.reset_index()
submission.to_csv('submission.csv', index = False)